In [ ]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression # cross_val_score를 위해 사용

# --------------------
# 데이터 로드 및 전처리
# --------------------

# 데이터 로드
df = pd.read_csv("health_survey.csv")

# 범주형 변수에 대한 원-핫 인코딩 수행
# smoker와 activity_level을 범주형으로 처리 (drop_first=True로 다중공선성 방지)
df_encoded = pd.get_dummies(df, columns=['smoker', 'activity_level'], drop_first=True)

# 수치형 변수 스케일링 (로지스틱 회귀 모델의 안정성 확보)
scaler = StandardScaler()
# age와 bmi만 스케일링하고, 나머지는 그대로 둡니다.
df_encoded[['age', 'bmi']] = scaler.fit_transform(df_encoded[['age', 'bmi']])

# --------------------
# 독립변수 식 정의 (Statsmodels formula 형식)
# --------------------
# 모든 인코딩된 특성을 포함하여 formula를 동적으로 생성합니다.
features = ' + '.join(col for col in df_encoded.columns if col not in ['disease'])
formula = f"disease ~ {features}"

print(f"--- 분석에 사용될 Logit Formula: {formula} ---")

In [ ]:
# --------------------
# 1) 교차 검증 전 코드 (단일 학습 및 통계 해석)
# --------------------

print("\n\n#####################################################")
print("## 1. 교차 검증 전 (단일 모델 학습 및 해석)")
print("#####################################################")

# 1단계: Statsmodels Logit 모델 학습
model_single_fit = logit(formula, data=df_encoded).fit()

# 2단계: 결과 요약 출력 (OLS와 유사한 표 형태)
print("\n--- [1] Logit Regression Summary Table ---")
print(model_single_fit.summary())

# 3단계: 오즈비 (Odds Ratio) 확인
# 해석의 편의성을 위해 가장 중요한 특성(BMI)과 나머지 특성들을 출력
print("\n--- [2] 회귀 계수와 오즈비 (Odds Ratio) ---")
odds_ratios = np.exp(model_single_fit.params)
odds_ratio_table = pd.DataFrame({
    'Coef. (회귀 계수)': model_single_fit.params,
    'Odds Ratio (오즈비)': odds_ratios
})
# BMI와 흡연 여부(smoker_1)만 출력
# print(odds_ratio_table.loc[['bmi', 'smoker_1', 'activity_level_1', 'activity_level_2']])

# 오즈비 값 해석 예시
print("\n[오즈비 해석: OR > 1 이면 질병 발생 가능성 증가]")
print(f"BMI OR: {np.exp(model_single_fit.params['bmi']):.4f}")

In [ ]:
# --------------------
# 2) 교차 검증 후 코드 (K-Fold를 통한 일반화 성능 평가)
# --------------------

print("\n\n#####################################################")
print("## 2. 교차 검증 후 (일반화 성능 평가)")
print("#####################################################")

# 종속변수/독립변수를 Scikit-learn 형식으로 준비
y = df_encoded['disease']
X = df_encoded.drop('disease', axis=1)

# 1단계: Stratified K-Fold 설정 (K=5)
# 분류 문제이므로 클래스 비율을 유지하는 StratifiedKFold 사용
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 2단계: Scikit-learn 로지스틱 회귀 모델 정의
model_cv = LogisticRegression(random_state=42)

# 3단계: cross_val_score 함수로 교차 검증 실행
cv_scores = cross_val_score(
    model_cv,       # Scikit-learn 모델
    X, y,           # 전체 데이터
    cv=skf,         # K-Fold 설정
    scoring='accuracy', # 평가 지표
    n_jobs=-1
)

# 4단계: 최종 결과 출력
mean_cv_accuracy = cv_scores.mean()
std_cv_accuracy = cv_scores.std()

print("\n--- K-Fold 교차 검증 (5-Fold) 결과 ---")
print(f"각 Fold의 정확도: {cv_scores}")
print(f"교차 검증 평균 정확도: {mean_cv_accuracy:.4f}")
print(f"정확도 표준 편차: {std_cv_accuracy:.4f}")

# 해석: 이 평균 정확도 값은 모델이 새로운 데이터에 대해 기대할 수 있는 성능을 객관적으로 나타냅니다.